In [1]:
import os
from dotenv import load_dotenv
from ec2_management.ec2_manager import EC2Manager

In [2]:
load_dotenv()
AWS_ACCESS_KEY = os.environ['AWS_LAB_ACCESS_KEY']
AWS_SECRET_ACCESS_KEY = os.environ['AWS_LAB_SECRET_ACCESS_KEY']
AWS_SESSION_TOKEN = os.environ['AWS_LAB_SESSION_TOKEN']
AWS_REGION = os.environ['AWS_REGION']
AWS_KEY_PAIR = os.environ['AWS_KEY_PAIR']

In [3]:
manager = EC2Manager(
    aws_access_key=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    aws_region=AWS_REGION,
    aws_key_pair=AWS_KEY_PAIR
)

In [4]:
# add rule to the default security group to open port 8080 and 22
manager.open_port(default_group=True, port_number=8080)
manager.open_port(default_group=True, port_number=22)

{'warning': 'rule already created, skipping action'}

In [21]:
# with open('flower-dependencies/server/server_startup.txt', 'r') as f:
#     server_startup_script = f.read()
server_startup_script = """#!/bin/bash
set -x
sudo yum -y install python3-pip
sudo yum -y install git
cd /home/ec2-user
git clone --depth 1 --filter=blob:none --no-checkout https://github.com/fedorTurchenko18/dsc-515.git
cd dsc-515/
git sparse-checkout init --cone
git sparse-checkout set flower-ec2/flower-dependencies/server
git checkout
cd flower-ec2/flower-dependencies/server
while read -r package; do
    pip install "$package"
done < requirements.txt
python3 server.py"""
# create and run the server ec2 instance
flower_server = manager.create_instance('server', startup_script=server_startup_script)

In [9]:
response = manager.client.describe_instances(InstanceIds=[flower_server[0].instance_id])
# Extract the public IP address from the response
server_public_ip_address = response['Reservations'][0]['Instances'][0]['PublicIpAddress']

In [13]:
# read client start-up script to modify the server public IP there
with open('flower-dependencies/client/client_startup.txt', 'r') as f:
    client_startup_script = [line for line in f.readlines()]
# modify the line
client_startup_script = [
    line.replace(line[line.find('=')+len('='):line.find('\\')], server_public_ip_address) \
        if 'export PUBLIC_IP' in line else line \
            for line in client_startup_script
]
# write new version of the script
with open('flower-dependencies/client/client_startup.txt', 'w') as f:
    f.writelines(client_startup_script)

In [ ]:
# create and run the clients ec2 instances
client_startup_script_path = os.path.abspath('flower-dependencies/server/client_startup.txt')
flower_clients = manager.create_instance('clients', startup_script_path=client_startup_script_path, n_instances=2)

In [3]:
manager.terminate_instance(instance_ids=[i['InstanceId'] for i in flower_clients['Instances']])

{'TerminatingInstances': [{'CurrentState': {'Code': 32,
    'Name': 'shutting-down'},
   'InstanceId': 'i-0156f5f0d72cf92a0',
   'PreviousState': {'Code': 16, 'Name': 'running'}},
  {'CurrentState': {'Code': 32, 'Name': 'shutting-down'},
   'InstanceId': 'i-0c16cf6a52199f91d',
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': 'fbb6da46-51f2-4772-9dfa-35802137da00',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fbb6da46-51f2-4772-9dfa-35802137da00',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '938',
   'date': 'Fri, 17 Nov 2023 19:55:51 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}